<a href="https://colab.research.google.com/github/Malikarbaaz/Deep_Learning_basic_implementation_with_hyperparameter_tuning/blob/main/CNN_Model_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create CNN Model and Optimize it using Keras Tuner**

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 4.5 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.7.0


In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
# from fashion_mnist i have to load particular data .... i dont need to download the data.
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()      # this is the step where my data is loaded

4431872/4422102 [==============================] - 0s 0us/step


**it will download all the images that are present in the form of train images and labels, similarly for test images and labels.**

In [7]:
# as our data is in the gray scale image so, 
# we will scale down the images between 0 to 1 where all the pixels converted between 0 to 1.
# It will help us to train the model quickly

train_images = train_images/255.0     # this is my maximum pixel images
test_images = test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

I need to resize this which it will be very very helpful for our CNN model because in the input layer we need to give that how many number of images i have, what is the pixels along with the 4 dimension.

In [9]:
# resize 
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):             # here i am creating function build_model and inside hp is my hyper parameter
  model = keras.Sequential([     # in order to create a CNN model we used keras.Sequential
    keras.layers.Conv2D(         # inside keras.Sequential i am adding Conv2D layer
        filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step =16),   # when i am adding Conv2D layer the 1st parameter i have to give is filters
# hp.Int is creating a range of integers with 'conv_1_filter' name with minimun and maximum value and it will try to find which Convolution layers
# give us the best performance between 32 to 128 [it will try all the layers between 32 to 128 and came up with best layer].      
        kernel_size = hp.Choice('conv_1_kernel', values = [3,5]), #kernel_size which is also called as filter_size and hp.Choice help us to select either 3 or 5.
        activation = 'relu',
        input_shape = (28,28,1) 
    ),
    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step =16),      # similarly second layer
        kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
        activation = 'relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step =16),       # flatten vector
        activation = 'relu'
    ),
    keras.layers.Dense(10, activation = 'softmax')   # we adding last dense layer with 10 outputs and activation we are using is softmax                                                 
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),    # same as it is explain above
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [11]:
from keras_tuner import RandomSearch  # it will try to find out which parameter will best for this CNN
from keras_tuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search = RandomSearch(build_model,                                    # calling my build_model function
                            objective= 'val_accuracy',                   
                            max_trials=5,                                # it will do a 5 different trials
                            directory = 'output',                        # all the checkpoints need to be same
                            project_name = 'Mnist Fashion')              # here I am giving a project name

In [13]:
tuner_search.search(train_images,train_labels, epochs = 3, validation_split = 0.1)         # it will search for best parameter for this above particular build model

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.9139999747276306

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 03m 09s
INFO:tensorflow:Oracle triggered exit


here I am giving a max_trial as 5 and epoch is 3 which means my model will run for 5 different trials with 3 epochs. [In every trial my model will run for 3 epochs and finally give us the best]

In [14]:
model = tuner_search.get_best_models(num_models = 1)[0]                            # just get me the top model

In [15]:
model.summary()                # to see which is the best model selected for this CNN model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        41520     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 96)                2654304   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2,697,754
Trainable params: 2,697,754
Non-trainable params: 0
_________________________________________________________________


## retrain all the images consider this paticular model

In [16]:
# retrain model
model.fit(train_images,train_labels, epochs=10, validation_split=0.1, initial_epoch=2)  #initial_epoch=2 means it will start from epoch 3 

Epoch 3/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1185 - accuracy: 0.9555 - val_loss: 0.2776 - val_accuracy: 0.9103
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0786 - accuracy: 0.9710 - val_loss: 0.3318 - val_accuracy: 0.9115
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0563 - accuracy: 0.9794 - val_loss: 0.3471 - val_accuracy: 0.9142
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0406 - accuracy: 0.9856 - val_loss: 0.4143 - val_accuracy: 0.9127
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0303 - accuracy: 0.9893 - val_loss: 0.4651 - val_accuracy: 0.9098
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0218 - accuracy: 0.9921 - val_loss: 0.5038 - val_accuracy: 0.9133
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0207 - accuracy: 0.9931 - val_loss: 0.5215 - val_accuracy: